In [18]:
import csv

import led_config_utils
import mesh_config
from funky_lights import wavefront, led_config

from importlib import reload
reload(mesh_config)

all_segments = {}
for config in mesh_config.mesh_configs:
    # if config.mesh != '../controller/mesh/legs.obj':
    #     continue
    uid = config.uid_offset
    mesh = wavefront.load_obj(config.mesh)
    segments = []
    for group in mesh.groups:
        data = led_config_utils.prepare_data(group)
        data_2d = led_config_utils.project_2d(data)
        clusters, labels = led_config_utils.cluster(
            data_2d, eps=config.cluster_eps, min_samples=3)
        points, length = led_config_utils.generate_led_positions(
            data, clusters, labels, start_offset=config.start_offset, end_offset=config.end_offset, leds_per_meter=30)
        print('Segment %s: length=%.1fm, num_clusters=%s, num_leds=%s' %
              (group.name, length, len(clusters), len(points)))
        if len(points) == 0:
            continue
        segment = led_config.Segment(
            uid=uid, name=group.name, points=points, num_leds=points.shape[0], length=length)
        segments.append(segment)
        all_segments[uid] = segment
        # led_config_utils.plot_segment(data, clusters, labels, points)
        uid += 1

    csv_header = ['uid', 'name', 'num_leds', 'length', 'reversed']
    csv_data = []
    for segment in segments:
        csv_data.append([segment.uid,
                        segment.name,
                        segment.num_leds,
                        segment.length,
                        False])

    with open(config.modelled_csv, 'w', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(csv_header)
        for row in csv_data:
            writer.writerow(row)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Segment dome/center: length=7.1m, num_clusters=32, num_leds=214
Segment dome/left: length=6.8m, num_clusters=43, num_leds=205
Segment dome/right: length=6.8m, num_clusters=43, num_leds=205
Segment trunk/Body1: length=5.0m, num_clusters=454, num_leds=150
Segment trunk/Body2: length=3.3m, num_clusters=281, num_leds=98
Segment trunk/Body3: length=3.0m, num_clusters=293, num_leds=91
Segment trunk/Body4: length=2.8m, num_clusters=259, num_leds=85
Segment trunk/Body5: length=2.6m, num_clusters=251, num_leds=79
Segment trunk/Body6: length=2.4m, num_clusters=217, num_leds=73
Segment trunk/Body7: length=2.2m, num_clusters=204, num_leds=67
Segment trunk/Body8: length=2.0m, num_clusters=157, num_leds=61
